In [1]:
from pyspark import *

In [2]:
sc = SparkContext(master="local", appName="test")

In [3]:
sc

<SparkContext master=local appName=test>

In [4]:
import urllib
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')
data_file = './data_10_percent.gz'

lines = sc.textFile(data_file)

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
sqlContext

In [7]:
type(data)

tuple

In [8]:
data = lines.map(lambda x: x.split(','))
data.take(5)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '235',
  '1337',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '29',
  '29',
  '1.00',
  '0.00',


In [9]:
type(data)

pyspark.rdd.PipelinedRDD

In [10]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [11]:
schema = StructType([
    StructField("duration", IntegerType(), True),
    StructField("protocol_type", StringType(), True),
    StructField("service", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("src_bytes", IntegerType(), True),
    StructField("dst_bytes", IntegerType(), True),
    StructField("interactions", StringType(), True),
    ])
schema

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [12]:
type(schema)

pyspark.sql.types.StructType

In [13]:
newData = data.map(lambda key:(int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))
newData.take(3)

[(0, 'tcp', 'http', 'SF', 181, 5450, 'normal.'),
 (0, 'tcp', 'http', 'SF', 239, 486, 'normal.'),
 (0, 'tcp', 'http', 'SF', 235, 1337, 'normal.')]

In [14]:
df = sqlContext.createDataFrame(newData, schema)
df

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [15]:
df.show(10)

+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s

In [16]:
df.createOrReplaceTempView("interactions")
sqlContext.sql("SELECT interactions FROM interactions").show()

+------------+
|interactions|
+------------+
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
+------------+
only showing top 20 rows



In [17]:
req = sqlContext.sql("SELECT duration,dst_bytes FROM interactions WHERE duration > 1000 AND dst_bytes == 0 ")
req.count()

139

In [18]:
type(req) 

pyspark.sql.dataframe.DataFrame

In [19]:
#req.take(req.count())

In [20]:
dataReq = req.rdd.map(lambda rowItem: "Duration: {}, Dest. bytes: {}".format(rowItem.duration, rowItem.dst_bytes))
dataReq.take(dataReq.count())

['Duration: 5057, Dest. bytes: 0',
 'Duration: 5059, Dest. bytes: 0',
 'Duration: 5051, Dest. bytes: 0',
 'Duration: 5056, Dest. bytes: 0',
 'Duration: 5051, Dest. bytes: 0',
 'Duration: 5039, Dest. bytes: 0',
 'Duration: 5062, Dest. bytes: 0',
 'Duration: 5041, Dest. bytes: 0',
 'Duration: 5056, Dest. bytes: 0',
 'Duration: 5064, Dest. bytes: 0',
 'Duration: 5043, Dest. bytes: 0',
 'Duration: 5061, Dest. bytes: 0',
 'Duration: 5049, Dest. bytes: 0',
 'Duration: 5061, Dest. bytes: 0',
 'Duration: 5048, Dest. bytes: 0',
 'Duration: 5047, Dest. bytes: 0',
 'Duration: 5044, Dest. bytes: 0',
 'Duration: 5063, Dest. bytes: 0',
 'Duration: 5068, Dest. bytes: 0',
 'Duration: 5062, Dest. bytes: 0',
 'Duration: 5046, Dest. bytes: 0',
 'Duration: 5052, Dest. bytes: 0',
 'Duration: 5044, Dest. bytes: 0',
 'Duration: 5054, Dest. bytes: 0',
 'Duration: 5039, Dest. bytes: 0',
 'Duration: 5058, Dest. bytes: 0',
 'Duration: 5051, Dest. bytes: 0',
 'Duration: 5032, Dest. bytes: 0',
 'Duration: 5063, De

In [21]:
from time import perf_counter

In [22]:
t1_start = perf_counter()  
sqlContext.sql("SELECT protocol_type, count('protocol_type') as count FROM interactions GROUP BY protocol_type").show()
t1_stop = perf_counter()  
runTimeQuery = t1_stop - t1_start
print("Times to execute the request : {0:.3f}s".format(runTimeQuery))

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Times to execute the request : 12.557s


In [23]:
#selectionner les colonnes duration, dst_bytes ou le protocole est tcp et les duration > 1000 avec des dst_bytes nuls
#df.filter(df.age > 3).collect()
#df.select(df.duration, df.dst_bytes).filter("protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0").collect()
t1_start = perf_counter()  
df.filter("duration > 1000 AND dst_bytes = 0").groupby("protocol_type").count().show()
t1_stop = perf_counter()  
runTimeQuery = t1_stop - t1_start
print("Requete executee en {0:.3f} secondes".format(runTimeQuery))

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Requete executee en 11.109 secondes


In [24]:
df.select(df.duration, df.dst_bytes).filter("protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0").collect()

[Row(duration=5057, dst_bytes=0),
 Row(duration=5059, dst_bytes=0),
 Row(duration=5051, dst_bytes=0),
 Row(duration=5056, dst_bytes=0),
 Row(duration=5051, dst_bytes=0),
 Row(duration=5039, dst_bytes=0),
 Row(duration=5062, dst_bytes=0),
 Row(duration=5041, dst_bytes=0),
 Row(duration=5056, dst_bytes=0),
 Row(duration=5064, dst_bytes=0),
 Row(duration=5043, dst_bytes=0),
 Row(duration=5061, dst_bytes=0),
 Row(duration=5049, dst_bytes=0),
 Row(duration=5061, dst_bytes=0),
 Row(duration=5048, dst_bytes=0),
 Row(duration=5047, dst_bytes=0),
 Row(duration=5044, dst_bytes=0),
 Row(duration=5063, dst_bytes=0),
 Row(duration=5068, dst_bytes=0),
 Row(duration=5062, dst_bytes=0),
 Row(duration=5046, dst_bytes=0),
 Row(duration=5052, dst_bytes=0),
 Row(duration=5044, dst_bytes=0),
 Row(duration=5054, dst_bytes=0),
 Row(duration=5039, dst_bytes=0),
 Row(duration=5058, dst_bytes=0),
 Row(duration=5051, dst_bytes=0),
 Row(duration=5032, dst_bytes=0),
 Row(duration=5063, dst_bytes=0),
 Row(duration=

In [25]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

def attack_or_normal_func(s):
    return "normal" if s == "normal." else "attack"

In [26]:
udf_attOrNorm = F.udf(attack_or_normal_func, StringType())

t1_start = perf_counter()  

dfbis = df.withColumn('label', udf_attOrNorm(df.interactions))
dfbis.show()

t1_stop = perf_counter()  
runTimeQuery = t1_stop - t1_start
print("Requete executee en {0:.3f} secondes".format(runTimeQuery))

+--------+-------------+-------+----+---------+---------+------------+------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions| label|
+--------+-------------+-------+----+---------+---------+------------+------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|normal|
|       0|          tcp|   http|  SF|      239|      486|     normal.|normal|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|normal|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|normal|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|normal|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|normal|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|normal|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|normal|
|       0|          tcp|   http|  SF|      210|      151|     normal.|normal|
|       0|          tcp|   http|  SF|      212|      786|     no

In [27]:
dfbis.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- interactions: string (nullable = true)
 |-- label: string (nullable = true)



In [28]:
t1_start = perf_counter()  

dfbis.groupby("label").count().show()

t1_stop = perf_counter()  
runTimeQuery = t1_stop - t1_start
print("Requete executee en {0:.3f} secondes".format(runTimeQuery))

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Requete executee en 10.841 secondes


In [29]:
t1_start = perf_counter()  

dfbis.groupby("label", "protocol_type").count().show()

t1_stop = perf_counter()  
runTimeQuery = t1_stop - t1_start
print("Requete executee en {0:.3f} secondes".format(runTimeQuery))

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Requete executee en 10.897 secondes
